## Teoría de perturbaciones

Consiste en resolver un sistema perturbado(se conoce la solución al no perturbado), y donde el interés es conocer la contribución de la parte perturbada $H'$ al nuevo sistema total. 

$$ H = H^{0} + H'$$

Para sistemas no degenerados, la corrección a la energía a primer orden se calcula como 

$$ E_{n}^{(1)} = \int\psi_{n}^{(0)*} H' \psi_{n}^{(0)}d\tau$$

Y la corrección a la función de onda, también a primer orden, se obtiene como:

 $$ \psi_{n}^{(1)} = \sum_{m\neq n} \frac{\langle\psi_{m}^{(0)} \left| H' \right| \psi_{n}^{(0)} \rangle}{E_{n}^{(0)} - E_{m}^{(0)}} \psi_{m}^{(0)}$$

** Tarea 1 : Programar ecuación para corrección de energía a primer orden si conoces $H^{0}$ y sus soluciones. **

**Tarea 2: Programar ecuación para correcciones a las funciones de onda a primer orden si conoces $H^{0}$ y sus soluciones. **

**Tarea 3: Investigue las soluciones a segundo orden y también programe las soluciones. **

La corrección de segundo orden a la energía se obtiene de la siguiente forma:

 $$ E_{n}^{(2)} = \sum_{m\neq n} \frac{\left| \langle\psi_{m}^{(0)} \left| H' \right| \psi_{n}^{(0)} \rangle\right|^{2}}{E_{n}^{(0)} - E_{m}^{(0)}}$$


In [1]:
### Solución

from sympy import *
from sympy import init_printing; init_printing(use_latex = 'mathjax')
from sympy.physics.qho_1d import E_n, psi_n
from scipy import constants
import numpy as np
var('x a l psi') #Variables a utilizar
var('m n n1 n2 l1 l2 hbar omega c d q r r1 r2 k', positive=True, real=True)

def get_corrections(PType,Pert,NI,Mass,L,AngFreq, Range, PertLim1, PertLim2):
    E_correction=0
    Psi_correction_sym=0
    E_correction2=0
    Psin=zeros(1,Range[1]-Range[0]+1)
    En=zeros(1,Range[1]-Range[0]+1)
    
    #Problem type original energies and wavefuncions
    if PType == "Harmonic oscilator":
        Psi=psi_n(n,x,m,omega)
        Eb=E_n(n,omega)
        for i in range (0,Range[1]-Range[0]+1):
            Psin[i]=psi_n(Range[0]+i,x,eval(Mass),eval(AngFreq))
            En[i]=E_n(Range[0]+i,eval(AngFreq))
    
    elif PType == 'Particle in a one-dimensional box':
        Psi=sqrt(2/a)*sin(n*pi*x/a)
        Eb=hbar**2*pi**2*n**2/(2*m*a**2)
        for i in range (0,Range[1]-Range[0]+1):
            Psin[i]=Psi.subs(n,Range[0]+i).subs(a,eval(L))
            En[i]=Eb.subs(n,Range[0]+i).subs(m,eval(Mass)).subs(a,eval(L))        
    
    #Solutions
    if (NI == 0 or Range[0]==0) and PType == 'Particle in a one-dimensional box':
        print("Error! n cannot be 0 for the particle in a box and helium atom problem")
        E_correction1=sympify('ERROR')
        E_correction2=sympify('ERROR')
        Psi_correction_sym=sympify('ERROR')
            
    else:
        PsiIntegrals=integrate(Psin[NI-Range[0]]*eval(Pert)*Psin, (x,eval(PertLim1),eval(PertLim2)))
        E_correction1=PsiIntegrals[NI-Range[0]]
        for i in range (0,Range[1]-Range[0]+1):
            if i!= NI-Range[0]:
                Psi_correction_sym=Psi_correction_sym+Psin[i]*PsiIntegrals[i]/(En[NI-Range[0]]-En[i])
                E_correction2=E_correction2+PsiIntegrals[i]**2/(En[NI-Range[0]]-En[i])
   
    
    return Psi, Eb, Psi_correction_sym, E_correction1, E_correction2
    


In [2]:
import ipywidgets as wid
from IPython.display import display
from ipywidgets import Layout, Box, Text, Dropdown, Label, IntSlider, IntRangeSlider, HTML

form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between'
)
PType=Dropdown(options=['Particle in a one-dimensional box', 'Harmonic oscilator'])
Pert=Text(value='0')
NI=IntSlider(value=0, min=0, max=9, step=1)
NI2=IntSlider(value=0, min=0, max=9, step=1)
lI1=IntSlider(value=0, min=0, max=9, step=1)
lI2=IntSlider(value=0, min=0, max=9, step=1)
Ran=IntRangeSlider(value=[0,9],min=0,max=9)
Mass=Text(value='m')
L=Text(value='a')
AngFreq=Text(value='omega')
Label1=HTML(value="Select <b>n</b> for the energy and wavefunction corrections, for particle in a box corrections the"
            " lowest <b>n</b> is <b>1</b>")
Label2=HTML(value="Select the function range for the wavefunction correction sum, for particle in a box the lowest" 
            " <b>m</b> is <b>1</b>. Remember <b>n</b> must be contained in the selected <b>m</b> range")
Label3=HTML(value="The following indexes only work for the Helium Atom problem, <b>n</b> index above is used as <b>n1</b>")
Label4=HTML(value="Select <b>l</b> for the energy and wavefunction corrections to the helium atom, remember that <b>l</b>"
            " must be smaller than <b>n</b>")
PertLim1=Text(value='0')
PertLim2=Text(value='a')

form_items1 = [
    Box([Label(value='Problem'),
         PType], layout=form_item_layout),
    Box([Label(value='Perturbation'),
         Pert], layout=form_item_layout),
    Box([Label(value='P.Min.Lim'),
         PertLim1], layout=form_item_layout),
    Box([Label(value='P.May.Lim'),
         PertLim2], layout=form_item_layout),
    Box([Label(value='P. Mass'),
         Mass], layout=form_item_layout),
    Box([Label(value='Box size'),
         L], layout=form_item_layout),
    Box([Label(value='Ang. Freq.'),
         AngFreq], layout=form_item_layout),
    Box([Label(value=''),
         Label1], layout=form_item_layout),
    Box([Label(value='n Index'),
         NI], layout=form_item_layout),
    Box([Label(value=''),
         Label2], layout=form_item_layout),
    Box([Label(value='m Range'),
         Ran], layout=form_item_layout)]

form1 = Box(form_items1, layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid 2px',
    align_items='stretch',
    width='40%'
))
form1


A Jupyter Widget

In [3]:
Psi,E,Psicorrection,Ecorrection,Ecorrection2=get_corrections(PType.value, Pert.value, NI.value, Mass.value, L.value, 
                                                             AngFreq.value, Ran.value, PertLim1.value, PertLim2.value)
Correction=Matrix([[sympify('psi_n(0)'),Psi],
                   [sympify('E_n(0)'),E],
                   [sympify('psi_%d(1)'  %(NI.value)),factor(Psicorrection)],
                   [sympify('E_%d(1)'  %(NI.value)),factor(Ecorrection)],
                   [sympify('E_%d(2)'  %(NI.value)),factor(Ecorrection2)]])
Correction

⎡                                                      2                      
⎢                      -n                        -m⋅ω⋅x                       
⎢                      ───                 ____  ────────                     
⎢                       2  4 ___ 4 ___    ╱ 1      2⋅ℏ           ⎛   √m⋅√ω⋅x⎞ 
⎢                     2   ⋅╲╱ m ⋅╲╱ ω ⋅  ╱  ── ⋅ℯ        ⋅hermite⎜n, ───────⎟ 
⎢                                      ╲╱   n!                   ⎝      √ℏ  ⎠ 
⎢ψₙ(0)                ─────────────────────────────────────────────────────── 
⎢                                           4 ___ 4 ___                       
⎢                                           ╲╱ ℏ ⋅╲╱ π                        
⎢                                                                             
⎢Eₙ(0)                                     ℏ⋅ω⋅(n + 1/2)                      
⎢                                                                             
⎢                                                   

**Tarea 4.  Resolver el átomo de helio aplicando los programas anteriores.** 

Para resolver el átomo de helio se toma el átomo de hidrógeno como base en donde se considera que el Hamiltoniano original es:

$$ H_{0} = H_{1} + H_{2}$$

Y la función sin perturbar es:

$$ \psi_{n_1 l_1 n_2 l_2} = \psi_{n_1 l_1}(r_1)\psi_{n_2 l_2}(r_2) $$

Para lo cual la energía se obtiene de la siguiente forma:

$$ E_{n_1  n_2} = -\alpha \left( \frac{1}{n_1}+\frac{1}{n_2} \right) $$

Ahora agregaremos la perturbación que representa el término repulsivo:

$$ H'= \frac{e^2}{\left|r_1-r_2\right|}$$


In [4]:
from sympy.physics.hydrogen import E_nl, R_nl
def Helium(N1,N2,L1,L2):
    
    Eb=E_nl(N1,1)+E_nl(N2,1)
    Psi1=R_nl(N1, L1, r1, Z=1)
    Psi2=R_nl(N2, L2, r2, Z=1)
    Psi=Psi1*Psi2
    E_correction1=integrate(integrate(r1**2*r2**2*conjugate(Psi1)*conjugate(Psi2)*q**2*Psi1*Psi2/abs(r1-r2), (r1,0,oo)), (r2,0,oo))
    E_correctionR1=q**2*integrate(r2**2*conjugate(Psi2)*Psi2*(integrate(r1**2*conjugate(Psi1)*Psi1/r2, (r1,0,r2))+integrate(r1**2*conjugate(Psi1)*Psi1/r1, (r1,r2,oo))), (r2,0,oo))
    
    return Psi, Eb, E_correction1, E_correctionR1

In [5]:
NI1=IntSlider(value=1, min=0, max=9, step=1)
NI2=IntSlider(value=1, min=0, max=9, step=1)
LI1=IntSlider(value=0, min=0, max=9, step=1)
LI2=IntSlider(value=0, min=0, max=9, step=1)
Label3=HTML(value="Select <b>n1</b>, <b>n2</b>, <b>l1</b>, <b>l2</b> indexes to be used for the first order"
            " energy correction to the Helium Atom")

form_items2 = [
    Box([Label(value=''),
         Label3], layout=form_item_layout),
    Box([Label(value='n1 Index'),
         NI1], layout=form_item_layout),
    Box([Label(value='n2 Index'),
         NI2], layout=form_item_layout),
    Box([Label(value='l1 Index'),
         LI1], layout=form_item_layout),
    Box([Label(value='l2 Index'),
         LI2], layout=form_item_layout)]

form2 = Box(form_items2, layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid 2px',
    align_items='stretch',
    width='40%'
))
form2


A Jupyter Widget

In [7]:
HPsi,HE,HEcorrection, HER=Helium(NI1.value, NI2.value, LI1.value, LI2.value)

HCorrection=Matrix([[sympify('psi_(0)'), HPsi], 
                    [sympify('E_(0)'), HE], 
                    [sympify('E_(1)'), factor(HEcorrection)],
                    [sympify('E_(1)'), HER]])
HCorrection

⎡                         -r₁  -r₂                ⎤
⎢ψ(0)                  4⋅ℯ   ⋅ℯ                   ⎥
⎢                                                 ⎥
⎢E(0)                      -1                     ⎥
⎢                                                 ⎥
⎢            ∞                                    ⎥
⎢            ⌠                                    ⎥
⎢            ⎮            ∞                       ⎥
⎢            ⎮            ⌠                       ⎥
⎢            ⎮            ⎮   2  -2⋅r₁            ⎥
⎢          2 ⎮   2  -2⋅r₂ ⎮ r₁ ⋅ℯ                 ⎥
⎢E(1)  16⋅q ⋅⎮ r₂ ⋅ℯ     ⋅⎮ ────────── d(r₁) d(r₂)⎥
⎢            ⎮            ⎮ │r₁ - r₂│             ⎥
⎢            ⎮            ⌡                       ⎥
⎢            ⎮            0                       ⎥
⎢            ⌡                                    ⎥
⎢            0                                    ⎥
⎢                                                 ⎥
⎢                            2                    ⎥
⎢           

**Tarea 5: Método variacional-perturbativo. **


Este método nos permite estimar de forma precisa $E^{(2)}$ y correcciones perturbativas de la energía de órdenes más elevados para el estado fundamental del sistema, sin evaluar sumas infinitas. Ver ecuación 9.38 del libro. 

$$ \langle u \left| H^0 - E_g^{(0)} \right| u \rangle + \langle u \left| H^\prime - E_g^{(1)} \right| \psi_{g}^{(0)} \rangle 
+  \langle \psi_{g}^{(0)} \left| H^\prime - E_g^{(1)} \right| u \rangle \geq E_g^{(2)} $$

In [9]:
def var_pert(PType,u,Pert):
    #Problem type original energies and wavefuncions
    if PType == "Harmonic oscilator":
        Psin=psi_n(n,x,m,omega)
        En=E_n(n,omega)
        Psig=psi_n(0,x,m,omega)
        Eg=E_n(0,omega)
        H=(-hbar**2/(2*m))*diff(u, x, 2) + (m*omega**2*x**2/2)*u
        
        E_correction1=integrate(Psig*Pert*Psig, (x,-oo,oo))
        EQ=integrate(u*(H+Eg)*u, (x,-oo,oo))+integrate(u*(Pert-E_correction1)*Psig, (x,-oo,oo))+integrate(Psig*(Pert-E_correction1)*u, (x,-oo,oo))
        Opt=solve(diff(EQ,k,1),k, dict=true)
        EQ=EQ.subs(k,Opt)
    
    elif PType == 'Particle in a one-dimensional box':
        Psin=sqrt(2/a)*sin(n*pi*x/a)
        En=hbar**2*pi**2*n**2/(2*m*a**2)
        Psig=Psin.subs(n,1)
        Eg=En.subs(n,1)
        H=(-hbar**2/(2*m))*diff(u, x, 2)
            
        E_correction1=integrate(Psig*Pert*Psig, (x,0,a))
        EQ=integrate(u*H-u*Eg*u, (x,0,a))+integrate(u*(Pert-E_correction1)*Psig, (x,0,a))+integrate(Psig*(Pert-E_correction1)*u, (x,0,a))
        Opt=solve(diff(EQ,k,1),k, dict=true)
        EQ=EQ.subs(k,Opt[0][k])
    return Psin, En, E_correction1, EQ

In [11]:
PType2=Dropdown(options=['Particle in a one-dimensional box', 'Harmonic oscilator'])
fPrueba=Text(value='x*k')
Pert2=Text(value='0')
Label4=HTML(value="Select the problem type and write the perturbation and test function, include a constant k for "
            "which the system will be optimized")

form_items3 = [
    Box([Label(value=''),
         Label4], layout=form_item_layout),
    Box([Label(value='Problem'),
         PType2], layout=form_item_layout),
    Box([Label(value='Test fun.'),
         fPrueba], layout=form_item_layout), 
    Box([Label(value='Perturbation'),
         Pert2], layout=form_item_layout)]

form3 = Box(form_items3, layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid 2px',
    align_items='stretch',
    width='40%'
))
form3


A Jupyter Widget

In [12]:
var_pert(PType2.value, eval(fPrueba.value), eval(Pert2.value))

⎛                                    ⎛    3     3⎞                            
⎜                                    ⎜   a     a ⎟                            
⎜                                  2⋅⎜- ──── + ──⎟                     2   4⋅a
⎜       ___              2   2  2    ⎜     2   6 ⎟     6   ⎛         2⎞       
⎜      ╱ 1     ⎛π⋅n⋅x⎞  π ⋅h̅ ⋅n     ⎝  4⋅π      ⎠    a ⋅m⋅⎝-33 + 4⋅π ⎠       
⎜√2⋅  ╱  ─ ⋅sin⎜─────⎟, ─────────, ───────────────, - ────────────────── + ───
⎜   ╲╱   a     ⎝  a  ⎠       2            a                   8   2           
⎝                         2⋅a ⋅m                           3⋅π ⋅h̅            

                 ⎛      4      4⎞⎞
3   ⎛         2⎞ ⎜  11⋅a    2⋅a ⎟⎟
 ⋅m⋅⎝-33 + 4⋅π ⎠⋅⎜- ───── + ────⎟⎟
                 ⎜      3   3⋅π ⎟⎟
                 ⎝   2⋅π        ⎠⎟
─────────────────────────────────⎟
            5     2              ⎟
           π ⋅a⋅h̅               ⎠

**Tarea 6. Revisar sección 9.7. **

Inicialmente a mano, y sengunda instancia favor de intentar programar sección del problema, i.e. integral de Coulomb  e integral de intercambio.

## Siguiente: Segunda parte, Octubre
Simetrías moleculares y Hartree-Fock

NotProgramed